<a href="https://colab.research.google.com/github/robocarstore/donkey-car-training-on-google-colab/blob/master/Donkey_Car_Training_using_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Donkey Car Training using Google Colab

Train your donkey car model using accelerated GPU for FREE on Google Colab

## Check GPU allocation
If "Found GPU at: / device: GPU: 0" is displayed, the GPU is ready to use.

In [1]:
import tensorflow
print(tensorflow.__version__)

2.9.2


## Git Clone the donkey repository

In [2]:
!git clone https://github.com/robocarstore/donkeycar.git 
%cd /content/donkeycar

!git checkout robocarstore_v43

Cloning into 'donkeycar'...
remote: Enumerating objects: 15076, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 15076 (delta 4), reused 8 (delta 0), pack-reused 15058
Receiving objects: 100% (15076/15076), 99.56 MiB | 34.91 MiB/s, done.
Resolving deltas: 100% (9669/9669), done.
/content/donkeycar
Branch 'robocarstore_v43' set up to track remote branch 'robocarstore_v43' from 'origin'.
Switched to a new branch 'robocarstore_v43'


## Install donkey car

In [3]:
!pip3 install -e .[pc]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/donkeycar
     |████████████████████████████████| 99 kB 4.6 MB/s 
     |████████████████████████████████| 865 kB 33.6 MB/s 
     |████████████████████████████████| 90 kB 8.8 MB/s 
     |████████████████████████████████| 29.5 MB 1.4 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=1ebfba41714d1fb5ac99aca8271712164aa31f0cb59b9feb91cc35bce2faab36
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62133 sha256=81bf229a884a1c5646cab8afc92864dab854d1fdb17b9b2b59d76321eb2f84cb
  Stored in directory: /root/.cache/pip/wheels/d0/bf/ac/2b3f43f8c6fcd0f4ba5395397458c521eb0b52d33b574a5a40
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9632 sha256=4519be103ecb1a40347eb19ec56

## Create Project

In [4]:
!donkey createcar --path /content/mycar

________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.22 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Copying car application template: complete
Copying car config defaults. Adjust these before starting your car.
Copying train script. Adjust these before starting your car.
Copying calibrate script. Adjust these before starting your car.
Copying my car config overrides
Donkey setup complete.


## Prepare Data
### Copy the following code and run on pi


```bash
cd ~/mycar/data
tar -czf tub_xx_yyyy_mm_dd.tar.gz tub_xx_yyyy_mm_dd

```

This will create a tub_xx_yyyy_mm_dd.tar.gz file under ~/mycar/data

## Upload Data


### Copy the tub to your local pc

Run this on your local pc if you are using linux/mac
```
sftp pi@raspberry.local
cd ~/mycar/data
get tub_xx_yyyy_mm_dd.tar.gz
```

If you are on a windows, download sftp utility like [filezilla](https://filezilla-project.org/) or [putty](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html)

### Define your tub name here

In [5]:
tub_name="tub_13_22-10-11"

### Upload the tub to Google Drive

First upload the tub_x_yyyy_mm_dd.tar.gz to Google Drive. We will then mount Google Drive from colab and copy the data from Drive directly. When you run the cell below, you will need to click the link and generate an authorization code to for colab to access your drive.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Suppose you upload the tub_xx_yyyy_mm_dd.tar.gz to Google Drive/mycar/tub_xx_yyyy_mm_dd.tar.gz, this is how you copy it from Google Drive to colab

In [7]:
%cd /content/mycar/data
!cp /content/drive/My\ Drive/mycar/{tub_name}.tar.gz .

/content/mycar/data


And untar it to the right place

In [8]:
!tar -xzf {tub_name}.tar.gz

Check whether the data is there

In [9]:
!ls {tub_name}

catalog_0.catalog	    catalog_3.catalog_manifest
catalog_0.catalog_manifest  catalog_4.catalog
catalog_1.catalog	    catalog_4.catalog_manifest
catalog_1.catalog_manifest  images
catalog_2.catalog	    manifest.json
catalog_2.catalog_manifest  meta.json
catalog_3.catalog	    tub_13_22-10-11_hist.png


## Train your model

In [17]:
%cd /content/mycar

!donkey train --tub ./data/{tub_name} --model ./models/mypilot.h5

/content/mycar
________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.22 ...
INFO:donkeycar.config:loading config file: ./config.py
INFO:donkeycar.config:loading personal config over-rides from ./myconfig.py
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:donkeycar.utils:get_model_by_type: model type is: linear
2022-10-18 05:12:47.041135: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:donkeycar.parts.keras:Created KerasLinear with interpreter: KerasInterpre

Check if the model is generated



In [ ]:
!ls -alh /content/mycar/models

In [ ]:
%cd /content/mycar/models

import glob
file = glob.glob("*.png")
  
from IPython.display import Image
Image(file[0])

## Copy the trained model back to Donkey Car (Pi)

Once the training is complete on colab, download the model file under /content/mycar/models/ folder location.

In [ ]:
from google.colab import files

files.download('./mypilot.h5')

Alternatively, you can copy the model back to Google Drive too

In [ ]:
!cp /content/mycar/models/mypilot.h5 /content/drive/My\ Drive/mycar/models/mypilot.h5

### Copy the file from your PC or Mac to the Raspberry Pi using Filezilla or scp command.

```
sftp pi@raspberry.local
cd mycar/models
put mypilot.h5
```

## Start Autopilot on Pi


```bash
cd ~/mycar
python manage.py drive --model models/mypilot.h5 --js
```


## Bonus - Salient Object Visualization

In [18]:
# !pip install git+https://github.com/autorope/keras-vis.git
!pip uninstall keras-vis
!pip install git+https://github.com/sctse999/keras-vis
  
  

Found existing installation: keras-vis 0.4.1
Uninstalling keras-vis-0.4.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/keras_vis-0.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/vis/*
Proceed (y/n)? y
  Successfully uninstalled keras-vis-0.4.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/sctse999/keras-vis to /tmp/pip-req-build-s_noomjb
  Running command git clone -q https://github.com/sctse999/keras-vis /tmp/pip-req-build-s_noomjb
  Created wheel for keras-vis: filename=keras_vis-0.5.0-py2.py3-none-any.whl size=39002 sha256=5c47b22b3a21d8c2d1b14a6051d2c311e1780e3b7a8498aff19da8473e3922cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-r3bxbdmi/wheels/7a/b6/3c/2b5abdaedcd9bd808857637cbef303bac47848674bb98200b4
Successfully built keras-vis


In [20]:
!pip install imageio-ffmpeg


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.5 MB/s 


In [21]:
%cd /content/mycar
!donkey makemovie --tub data/{tub_name} --model models/mypilot.h5 --type linear --salient

/content/mycar
________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.22 ...
Traceback (most recent call last):
  File "/usr/local/bin/donkey", line 33, in <module>
    sys.exit(load_entry_point('donkeycar', 'console_scripts', 'donkey')())
  File "/content/donkeycar/donkeycar/management/base.py", line 616, in execute_from_command_line
    c.run(args[2:])
  File "/content/donkeycar/donkeycar/management/base.py", line 307, in run
    from donkeycar.management.makemovie import MakeMovie
  File "/content/donkeycar/donkeycar/management/makemovie.py", line 1, in <module>
    import moviepy.editor as mpy
  File "/us

In [ ]:
from google.colab import files

%cd /content/mycar
!ls -ahl
files.download('tub_movie.mp4')